In our tests, the longest running test was:

> [success] 21.88% tests.tests01_geom.tests03_core.Test15_DetectLensTor.test12_plot_Etend_AlongLOS: 459.2323s

Let's build a core example out of test 12.

In [1]:
!ls tests01_geom/

AUG_Ves.txt                     TFG_VesTor_AUG_Test_sh00000.npz
G_015.npz                       __init__.py
G_020.npz                       __init__.pyc
I_050.npz                       tests01_GG.py
I_052.npz                       tests01_GG.pyc
L_009.npz                       tests02_compute.py
L_010.npz                       tests02_compute.pyc
L_011.npz                       tests03_core.py
L_012.npz                       tests03_core.pyc
TFG_VesLin_AUG_Test_sh00000.npz


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tofu.defaults as tfd
import tofu.pathfile as tfpf
import tofu.geom as tfg

In [3]:
PVes = np.loadtxt(os.path.join('tests01_geom', 'AUG_Ves.txt'), dtype='float', skiprows=1, ndmin=2, comments='#')
DLong = 2.*np.pi*1.7*np.array([-0.5,0.5])

In [4]:
VesTor = tfg.Ves('Test', PVes, Type='Tor', shot=0, Exp='AUG', SavePath='')
VesLin = tfg.Ves('Test', PVes, Type='Lin', DLong=DLong, shot=0, Exp='AUG', SavePath='')

In [5]:
Ves = VesTor

In [8]:
# %load -s Test15_DetectLensTor tests01_geom/tests03_core.py
O = np.array([0.,Ves._P1Max[0],Ves._P2Max[1]])
nIn = np.array([0.,-1.,-1.])
Rad, F1 = 0.005, 0.010
Lens = tfg.Lens('Test', O, nIn, Rad, F1, Ves=Ves, Type='Sph', Exp='AUG', Diag='Test', shot=0, SavePath='')
Obj = tfg.Detect('Test', {'Rad':0.001}, Optics=Lens, Ves=Ves, Exp='AUG', Diag='Test', shot=0, SavePath='',
                     Etend_Method='quad', Etend_RelErr=1.e-1,
                     Cone_DRY=0.005, Cone_DXTheta=np.pi/512., Cone_DZ=0.005,
                    Calc=False)

TFG.Detect object Test : Creating...
TFG.Detect object Test : Created !


In [24]:
%time Obj._calc_All(CalcEtend=True, CalcCone=False, CalcPreComp=False, CalcSpanImp=False, Etend_Method='trapz')

    Test : Computing LOS...
    Test : Computing Entendue...
CPU times: user 1.31 s, sys: 108 ms, total: 1.42 s
Wall time: 1.43 s


In [14]:
%load_ext line_profiler

In [ ]:
help(lprun)

In [17]:
type(Obj)

tofu.geom._core.Detect

In [26]:
%lprun -f tfg._GG.Calc_SAngVect_LPolysPoints_Flex_Lens Obj._calc_All(CalcEtend=True, CalcCone=False, CalcPreComp=False, CalcSpanImp=False, Etend_Method='trapz')

/Users/kappamaki/anaconda/envs/py27-tofu/lib/python2.7/site-packages/line_profiler.py:294: UserWarning: Could not extract a code object for the object <built-in function Calc_SAngVect_LPolysPoints_Flex_Lens>
  profile = LineProfiler(*funcs)


    Test : Computing LOS...
    Test : Computing Entendue...


In [ ]:
"""Timer unit: 1e-06 s

Total time: 1.301 s
File: /Users/kappamaki/workspace/tofu/tofu/geom/_compute.py
Function: Calc_Etendue_PlaneLOS at line 523

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   523                                           def Calc_Etendue_PlaneLOS(Ps, nPs, DPoly, DBaryS, DnIn, LOPolys, LOnIns, LOSurfs, LOBaryS, SAngPlane, VPoly, VVin, DLong=None,
   524                                                   Lens_ConeTip=None, Lens_ConeHalfAng=None, RadL=None, RadD=None, F1=None, NEdge=TFD.DetSpanNEdge, NRad=TFD.DetSpanNRad,
   525                                                   OpType='Apert', VType='Tor', Mode='quad', epsrel=TFD.DetEtendepsrel, dX12=TFD.DetEtenddX12, dX12Mode=TFD.DetEtenddX12Mode, e1=None,e2=None, Ratio=0.02, Colis=TFD.DetCalcEtendColis, Details=False, Test=True):    # Used
   526                                               """ Computes the Etendue of a Detect with Apert on N given planes parametrised by (P,nP) with two possible methods : discrete (simps) or (dblquad)
   527                                           
   528                                               Inputs :
   529                                                   D           A Detect instance
   530                                                   P           A (3,N) np.ndarray corresponding to the cartesian coordinates of N points (one for each plane)
   531                                                   nP          A (3,N) np.ndarray corresponding to the cartesian coordinates of N normal vectors (one for each plane)
   532                                                   Mode        'dblquad' (default) or 'simps' or 'trapz' to choose the scipy computation method of the integral
   533                                                   epsrel      For 'dblquad', a positive float defining the relative tolerance allowed
   534                                                   dX12        For 'simps'/'trapz', a list of 2 floats defining the resolution of the sampling in X1 and X2
   535                                                   dX12Mode    For 'simps'/'trapz', 'rel' or 'abs', if 'rel' the resolution dX12 is in dimensionless units in [0;1] (hence a value of 0.1 means 10 discretisation points between the extremes), if 'abs' dX12 is in meters
   536                                                   e1          The e1 unitary vector (corresponds to X1), if unspecified, the optimal vector is used
   537                                                   e2          The e2 unitary vector (corresponds to X2), must be orthogonal to e1, if unspecified, the optimal vector is used
   538                                                   Ratio       A float specifying the relative margin to be taken for integration boundaries
   539                                               Outputs :
   540                                                   Etendue     A positive float, the calculated etendue (in )
   541                                                   e1          The e1 unitary vector (corresponds to X1) used
   542                                                   e2          The e2 unitary vector (corresponds to X2) used
   543                                                   err         An array containing error flags for the integral computation
   544                                               """
   545                                           
   546         1            4      4.0      0.0      if Test:
   547         1            4      4.0      0.0          assert isinstance(Ps,np.ndarray) and Ps.ndim==2 and Ps.shape[0]==3, "Arg Ps should be a (3,N) ndarray !"
   548         1            3      3.0      0.0          assert isinstance(nPs,np.ndarray) and nPs.shape==Ps.shape, "Arg nPs should be a (3,N) ndarray !"
   549         1            2      2.0      0.0          assert Mode in ['quad','simps','trapz'], "Arg Mode should be 'quad' or 'simps' or 'trapz' !"
   550                                           
   551         1            2      2.0      0.0      nPlans = nPs.shape[1]
   552         1           60     60.0      0.0      nPs = nPs/np.tile(np.sqrt(np.sum(nPs**2,axis=0)),(3,1))
   553         1          118    118.0      0.0      e1, e2 = GG.Calc_DefaultCheck_e1e2_PLanes_2D(Ps, nPs, e1, e2)
   554                                           
   555         1            2      2.0      0.0      if OpType=='Apert':
   556                                                   LnPtemp = np.asarray(LOnIns)*np.tile(LOSurfs,(3,1)).T
   557                                                   Out = Calc_ViewConePointsMinMax_PlanesDetectApert_2Steps(DPoly, DBaryS, LOPolys, LnPtemp, LOSurfs, LOBaryS[0], Ps, nPs, e1=e1, e2=e2, OpType=OpType, Lens_ConeTip=Lens_ConeTip, NEdge=NEdge, NRad=NRad, Test=False)
   558         1            2      2.0      0.0      elif OpType=='Lens':
   559         1          900    900.0      0.1          Out = Calc_ViewConePointsMinMax_PlanesDetectLens(LOPolys[0], Lens_ConeTip, Ps, nPs, e1=e1, e2=e2, Test=False)
   560                                           
   561         1            3      3.0      0.0      MinX1, MinX2, MaxX1, MaxX2 = Out[0], Out[1], Out[2], Out[3]
   562         1            7      7.0      0.0      MinX1 = MinX1 - Ratio*(MaxX1-MinX1)
   563         1            4      4.0      0.0      MaxX1 = MaxX1 + Ratio*(MaxX1-MinX1)
   564         1            5      5.0      0.0      MinX2 = MinX2 - Ratio*(MaxX2-MinX2)
   565         1            4      4.0      0.0      MaxX2 = MaxX2 + Ratio*(MaxX2-MinX2)
   566                                           
   567         1           11     11.0      0.0      Etend = np.nan*np.ones((nPlans,))
   568         1            6      6.0      0.0      err = np.nan*np.ones((nPlans,))
   569         1           11     11.0      0.0      NOPoly = len(LOPolys)
   570                                           
   571         1            3      3.0      0.0      PBary, nPtemp, e1bis, e2bis = SAngPlane
   572         1            3      3.0      0.0      LPolys = [DPoly] + LOPolys
   573                                           
   574         1            2      2.0      0.0      if Mode=='quad' and not Details:
   575                                                   LBaryS = np.ascontiguousarray(np.vstack(tuple([DBaryS]+LOBaryS)).T)
   576                                                   LnIn = np.ascontiguousarray(np.vstack(tuple([DnIn]+LOnIns)).T)
   577                                                   nPolys = NOPoly+1
   578                                                   for ii in range(0,nPlans):
   579                                                       if not np.any(np.isnan(np.array([MinX1[ii], MinX2[ii], MaxX1[ii], MaxX2[ii]]))):
   580                                                           def X2Min(x):
   581                                                               return MinX2[ii]
   582                                                           def X2Max(x):
   583                                                               return MaxX2[ii]
   584                                                           if OpType=='Apert':
   585                                                               FuncSA = GG.FuncSAquad_Apert(LPolys, nPtemp, e1bis, e2bis, Ps[:,ii], e1[:,ii], e2[:,ii], LBaryS, LnIn, nPolys, PBary, VPoly, VVin, DLong=DLong, VType=VType, Colis=Colis)
   586                                                           else:
   587                                                               thet = np.linspace(0.,2.*np.pi,LOPolys[0].shape[1])
   588                                                               FuncSA = GG.FuncSAquad_Lens(LOBaryS[0], Lens_ConeTip, Ps[:,ii], e1[:,ii], e2[:,ii], LBaryS, LnIn, nPolys,
   589                                                                       RadL, RadD, F1, np.tan(Lens_ConeHalfAng), LOPolys[0], VPoly, VVin, DLong=DLong, VType=VType, thet=thet, VectReturn=False, Colis=Colis)
   590                                           
   591                                                           Etend[ii], err[ii] = GG.dblquad_custom(FuncSA, MinX1[ii], MaxX1[ii], X2Min, X2Max, epsabs=0., epsrel=epsrel)
   592                                           
   593                                               else:
   594         1            2      2.0      0.0          if dX12Mode=='rel':
   595         1            5      5.0      0.0              NumX1 = [int(1./dX12[0])]*MinX1.size
   596         1            3      3.0      0.0              NumX2 = [int(1./dX12[1])]*MinX2.size
   597                                                   else:
   598                                                       NumX1 = [int((MaxX1[zz]-MinX1[zz])/dX12[0]) for zz in range(0,nPlans)]
   599                                                       NumX2 = [int((MaxX2[zz]-MinX2[zz])/dX12[1]) for zz in range(0,nPlans)]
   600         2            7      3.5      0.0          NumP = [NumX1[zz]*NumX2[zz] for zz in range(0,nPlans)]
   601         2            6      3.0      0.0          for ii in range(0,nPlans):
   602         1            8      8.0      0.0              SA = np.zeros((NumP[ii],))
   603         1           48     48.0      0.0              X1 = np.linspace(MinX1[ii],MaxX1[ii],NumX1[ii],endpoint=True).reshape((NumX1[ii],1))
   604         1           32     32.0      0.0              X2 = np.linspace(MinX2[ii],MaxX2[ii],NumX2[ii],endpoint=True).reshape((1,NumX2[ii]))
   605         1           38     38.0      0.0              x1 = np.dot(X1,np.ones((1,NumX2[ii]))).reshape((1,NumP[ii]))
   606         1           22     22.0      0.0              x2 = np.dot(np.ones((NumX1[ii],1)),X2).reshape((1,NumP[ii]))
   607                                           
   608         1          273    273.0      0.0              Pps = np.dot(Ps[:,ii:ii+1],np.ones((1,NumP[ii]))) + np.dot(e1[:,ii:ii+1],x1) + np.dot(e2[:,ii:ii+1],x2)
   609         1          273    273.0      0.0              ind = _Detect_isOnGoodSide(Pps, DBaryS, DnIn, LOBaryS, LOnIns, NbPoly=None, Log='all')
   610         1           12     12.0      0.0              if np.any(ind):
   611         1          266    266.0      0.0                  Ppsind = Pps[:,ind] if ind.sum()>1 else Pps[:,ind].reshape((Pps.shape[0],1))
   612         1            3      3.0      0.0                  if OpType=='Apert':
   613                                                               SA[ind] = GG.Calc_SAngVect_LPolysPoints_Flex(LPolys, Ppsind,  PBary, nPtemp, e1bis, e2bis)[0]      # Cython
   614                                                           else:
   615         1            6      6.0      0.0                      SA[ind] = GG.Calc_SAngVect_LPolysPoints_Flex_Lens(LOBaryS[0][0],LOBaryS[0][1],LOBaryS[0][2], Lens_ConeTip[0],Lens_ConeTip[1],Lens_ConeTip[2], LOnIns[0][0],LOnIns[0][1],LOnIns[0][2],
   616         1          126    126.0      0.0                                                                        np.ascontiguousarray(Ppsind[0,:]),np.ascontiguousarray(Ppsind[1,:]),np.ascontiguousarray(Ppsind[2,:]), RadL, RadD, F1, np.tan(Lens_ConeHalfAng),
   617         1            9      9.0      0.0                                                                        np.ascontiguousarray(LOPolys[0][0,:]),np.ascontiguousarray(LOPolys[0][1,:]),np.ascontiguousarray(LOPolys[0][2,:]),
   618         1      1298483 1298483.0     99.8                                                                        thet=np.linspace(0.,2.*np.pi,LOPolys[0].shape[1]), VectReturn=False)
   619         1           15     15.0      0.0              indPos = SA>0.
   620         1            2      2.0      0.0              if Colis and np.any(indPos):
   621                                                           PpsindPos = Pps[:,indPos] if indPos.sum()>1 else Pps[:,indPos].reshape((Pps.shape[0],1))
   622                                                           indC = GG.Calc_InOut_LOS_Colis(DBaryS, PpsindPos, VPoly, VVin,Forbid=True,Margin=0.1) if VType=='Tor' else GG.Calc_InOut_LOS_Colis_Lin(DBaryS, PpsindPos, VPoly, VVin, DLong)
   623                                                           indnul = indPos.nonzero()[0]
   624                                                           SA[indnul[~indC]] = 0.
   625                                           
   626         1            2      2.0      0.0              if Mode=='simps':
   627                                                           Etend[ii] = GG.dblsimps_custom(SA.reshape((NumX1[ii],NumX2[ii])),x=X1.flatten(),y=X2.flatten())
   628         1            2      2.0      0.0              elif Mode=='trapz':
   629         1          199    199.0      0.0                  Etend[ii] = GG.dbltrapz_custom(SA.reshape((NumX1[ii],NumX2[ii])),x=X1.flatten(),y=X2.flatten())
   630         1            2      2.0      0.0      if Details:
   631                                                   return Etend, e1, e2, err, SA, X1, X2, NumX1, NumX2
   632                                               else:
   633         1            2      2.0      0.0          return Etend, e1, e2, err"""

# Cython 

In [27]:
%load_ext cython

In [53]:
%%cython -a
import math
cimport numpy as np
ctypedef np.float_t DTYPE_t
import cython
import Polygon as plg
import numpy as np

cdef inline DTYPE_t Cabs(DTYPE_t x):
    return x if x >= 0.0 else -x

@cython.boundscheck(False)
@cython.wraparound(False)
cdef Calc_SAngVect_1Point1Poly_NoVect(DTYPE_t P0, DTYPE_t P1, DTYPE_t P2, DTYPE_t[::1] Poly0, DTYPE_t[::1] Poly1, DTYPE_t[::1] Poly2, DTYPE_t G0, DTYPE_t G1, DTYPE_t G2):
    """ Return solid angle subtended by (closed) Poly from Point """
    cdef Py_ssize_t NPoly = len(Poly0), ii = 0
    cdef DTYPE_t PG0 = G0-P0, PG1 = G1-P1, PG2 = G2-P2
    cdef DTYPE_t PGn = math.sqrt(PG0*PG0+PG1*PG1+PG2*PG2)
    cdef DTYPE_t PP10, PP11, PP12, PP20, PP21, PP22, PP1n, PP2n, ATan1, ATan2
    cdef DTYPE_t SAngs = 0. 
    for ii in xrange(0,NPoly-1):
        PP10, PP11, PP12 = Poly0[ii]-P0, Poly1[ii]-P1, Poly2[ii]-P2
        PP20, PP21, PP22 = Poly0[ii+1]-P0, Poly1[ii+1]-P1, Poly2[ii+1]-P2
        PP1n = math.sqrt(PP10*PP10+PP11*PP11+PP12*PP12)
        PP2n = math.sqrt(PP20*PP20+PP21*PP21+PP22*PP22)
        ATan1 = Cabs(PG0*(PP11*PP22-PP12*PP21) + PG1*(PP12*PP20-PP10*PP22) + PG2*(PP10*PP21-PP11*PP20))
        ATan2 = PGn*PP1n*PP2n + (PG0*PP10+PG1*PP11+PG2*PP12)*PP2n + (PG0*PP20+PG1*PP21+PG2*PP22)*PP1n + (PP10*PP20+PP11*PP21+PP12*PP22)*PGn
        sa = math.atan2(ATan1, ATan2)
        SAngs = SAngs + sa if sa>=0 else SAngs + sa + math.pi
    return 2.*SAngs

@cython.boundscheck(False)
@cython.wraparound(False)
cdef Calc_SAngVect_1Point1Poly_FromList(DTYPE_t P0, DTYPE_t P1, DTYPE_t P2, list Poly, DTYPE_t G0, DTYPE_t G1, DTYPE_t G2):
    # Return solid angle subtended by Poly from Point, as well as normalised vector from Points to center of mass of Poly
    cdef Py_ssize_t NPoly = len(Poly), ii = 0
    cdef DTYPE_t PG0 = G0-P0, PG1 = G1-P1, PG2 = G2-P2
    cdef DTYPE_t PGn = math.sqrt(PG0*PG0+PG1*PG1+PG2*PG2)
    cdef DTYPE_t PP10, PP11, PP12, PP20, PP21, PP22, PP1n, PP2n, ATan1, ATan2
    cdef DTYPE_t SAngs=0., V0=PG0/PGn, V1=PG1/PGn, V2=PG2/PGn 
    for ii in xrange(0,NPoly-1):
        PP10, PP11, PP12 = Poly[ii][0]-P0, Poly[ii][1]-P1, Poly[ii][2]-P2
        PP20, PP21, PP22 = Poly[ii+1][0]-P0, Poly[ii+1][1]-P1, Poly[ii+1][2]-P2
        PP1n = math.sqrt(PP10*PP10+PP11*PP11+PP12*PP12)
        PP2n = math.sqrt(PP20*PP20+PP21*PP21+PP22*PP22)
        ATan1 = Cabs(PG0*(PP11*PP22-PP12*PP21) + PG1*(PP12*PP20-PP10*PP22) + PG2*(PP10*PP21-PP11*PP20))
        ATan2 = PGn*PP1n*PP2n + (PG0*PP10+PG1*PP11+PG2*PP12)*PP2n + (PG0*PP20+PG1*PP21+PG2*PP22)*PP1n + (PP10*PP20+PP11*PP21+PP12*PP22)*PGn
        sa = math.atan2(ATan1, ATan2)
        if sa>=0:
            SAngs = SAngs + sa
        else:
            SAngs = SAngs + sa + np.pi
    return 2.*SAngs, V0,V1,V2

@cython.wraparound(False)
@cython.boundscheck(False)
cdef Calc_SAngVect_LPolysPoints(list LPolys, np.ndarray[DTYPE_t, ndim=2] Points, np.ndarray[DTYPE_t, ndim=1,mode='c'] P, np.ndarray[DTYPE_t, ndim=1,mode='c'] nP, np.ndarray[DTYPE_t, ndim=1,mode='c'] e1P, np.ndarray[DTYPE_t, ndim=1,mode='c'] e2P):
    # Returns homothetic (with centers As) projections of Poly on plane (P,nP), and optionnaly their components (X1,X2) along (e1P,e2P)
    # Arbitrary Points and plane, but a unique plane and polygon list common to all of them
    #t1, t2, t3, t4, t5, t6 = 0.,0.,0.,0.,0.,0.    # DB
    #tt = dtm.datetime.now()                        # DB
    cdef Py_ssize_t NP = Points.shape[1], NList = len(LPolys)
    cdef Py_ssize_t ii, jj, indOKii
    cdef DTYPE_t P0=P[0], P1=P[1], P2=P[2], e10=e1P[0], e11=e1P[1], e12=e1P[2], e20=e2P[0], e21=e2P[1], e22=e2P[2]
    cdef np.ndarray[DTYPE_t, ndim=1, mode='c']  SAng = np.zeros((NP,))
    cdef np.ndarray[DTYPE_t, ndim=2]  Vect = np.nan*np.ones((3,NP))
    cdef np.ndarray[np.int_t, ndim=1, mode='c'] NPperPoly = np.array([LPolys[ii].shape[1] for ii in range(0,NList)],dtype=int)
    cdef np.ndarray[DTYPE_t, ndim=2]  Polys = np.concatenate(tuple(LPolys),axis=1)
    cdef Py_ssize_t                   NPoly = Polys.shape[1]
    cdef np.ndarray[DTYPE_t, ndim=3]  Asbis = np.tile(Points.reshape((3,1,NP)),(1,NPoly,1)).T
    cdef np.ndarray[DTYPE_t, ndim=3]  AM = np.tile(Polys.T.reshape(1,NPoly,3),(NP,1,1)) - Asbis
    cdef np.ndarray[DTYPE_t, ndim=3]  Pbis = np.tile(P.reshape(1,1,3),(NP,NPoly,1))
    cdef np.ndarray[DTYPE_t, ndim=3]  nPbis = np.tile(nP.reshape(1,1,3),(NP,NPoly,1))
    cdef np.ndarray[DTYPE_t, ndim=3]  ScaAMn = np.sum(AM*nPbis,axis=2,keepdims=True)
    cdef np.ndarray[DTYPE_t, ndim=3]  ScaAPn = np.sum((Pbis-Asbis)*nPbis,axis=2,keepdims=True)
    cdef np.ndarray indneg = np.any(ScaAMn*ScaAPn<0,axis=1).flatten()
    cdef Py_ssize_t[::1]              IndOK = (~indneg).nonzero()[0]
    cdef np.ndarray[DTYPE_t, ndim=3]  k = np.zeros((NP,NPoly,1))
    cdef np.ndarray[DTYPE_t, ndim=3]  e1bis = np.tile(e1P.reshape(1,1,3),(NP,NPoly,1))
    cdef np.ndarray[DTYPE_t, ndim=3]  e2bis = np.tile(e2P.reshape(1,1,3),(NP,NPoly,1))
    cdef np.ndarray[DTYPE_t, ndim=2]  PolyProjX1, PolyProjX2
    cdef DTYPE_t Bs0, Bs1
    cdef list Polyint, PX, PolyInt
    cdef list IndPoly = [(np.sum(NPperPoly[0:jj+1])-NPperPoly[jj], np.sum(NPperPoly[0:jj+1])-1) for jj in range(0,NList)]
    #t1 = (dtm.datetime.now()-tt).total_seconds()      # DB
    #tt = dtm.datetime.now()         # DB
    # Get the intersection of projected polygons for each point
    ind = np.abs(ScaAMn)-1.e-14 > 0.
    k[ind] = ScaAPn[ind]/ScaAMn[ind]
    k[indneg,:,:] = np.nan
    cdef np.ndarray[DTYPE_t, ndim=3] PDiff = Asbis + (np.tile(k.T,(3,1,1)).T)*AM - Pbis # = PolyProj - Pbis
    PolyProjX1 = np.sum(PDiff*e1bis,axis=2,keepdims=False)
    PolyProjX2 = np.sum(PDiff*e2bis,axis=2,keepdims=False)
    PX = [np.array([PolyProjX1[:,IndPoly[jj][0]:IndPoly[jj][1]+1],PolyProjX2[:,IndPoly[jj][0]:IndPoly[jj][1]+1]]).T for jj in range(0,NList)]
    #t2 = (dtm.datetime.now() - tt).total_seconds()    # DB
    # Compute solid angle for each point with non-zero intersection
    #tt3 = dtm.datetime.now()         # DB
    for ii in xrange(0,IndOK.size):
        #tt = dtm.datetime.now()         # DB
        indOKii = IndOK[ii]
        Polyint = [plg.Polygon(PX[0][:,indOKii,:])]
        for jj in range(1,NList):
            Polyint[0] = Polyint[0] & plg.Polygon(PX[jj][:,indOKii,:])
        #t4 += (dtm.datetime.now()-tt).total_seconds()   # DB
        if Polyint[0].area()>1e-12:
            #tt = dtm.datetime.now()    # DB
            Bs0, Bs1 = Polyint[0].center()
            PolyInt = [(P0+e10*pp[0]+e20*pp[1],P1+e11*pp[0]+e21*pp[1],P2+e12*pp[0]+e22*pp[1]) for pp in Polyint[0][0]+[Polyint[0][0][0]]]
            #t5 += (dtm.datetime.now()-tt).total_seconds()   # DB
            #tt = dtm.datetime.now()    # DB 
            SAng[indOKii], Vect[0,indOKii], Vect[1,indOKii], Vect[2,indOKii] = Calc_SAngVect_1Point1Poly_FromList(Points[0,indOKii],Points[1,indOKii],Points[2,indOKii], PolyInt, P0+e10*Bs0+e20*Bs1, P1+e11*Bs0+e21*Bs1, P2+e12*Bs0+e22*Bs1)
            #t6 += (dtm.datetime.now()-tt).total_seconds()   # DB
    #t3 = (dtm.datetime.now() - tt3).total_seconds()    # DB
    #print t1, t2, t3, t4, t5, t6 # DB
    return SAng, Vect, indneg

def LensDetect_isInside(DTYPE_t O0, DTYPE_t O1, DTYPE_t O2, DTYPE_t Tip0, DTYPE_t Tip1, DTYPE_t Tip2, DTYPE_t nIn0, DTYPE_t nIn1, DTYPE_t nIn2, 
        np.ndarray[DTYPE_t, ndim=1,mode='c'] Pt0, np.ndarray[DTYPE_t, ndim=1,mode='c'] Pt1, np.ndarray[DTYPE_t, ndim=1,mode='c'] Pt2, DTYPE_t tanthetmax):
    """ Return an array of bool indicating whether the provided points lie inside the viewing cone of the Detect+Lens system """
    cdef np.ndarray[DTYPE_t, ndim=1,mode='c'] din = (Pt0-O0)*nIn0 + (Pt1-O1)*nIn1 + (Pt2-O2)*nIn2
    cdef np.ndarray[DTYPE_t, ndim=1,mode='c'] dinTip = (Pt0-Tip0)*nIn0 + (Pt1-Tip1)*nIn1 + (Pt2-Tip2)*nIn2
    cdef np.ndarray[DTYPE_t, ndim=1,mode='c'] r = np.sqrt((Pt0-Tip0-dinTip*nIn0)**2 + (Pt1-Tip1-dinTip*nIn1)**2 + (Pt2-Tip2-dinTip*nIn2)**2)
    return (r/dinTip <= tanthetmax) & (din>0.)

def _Lens_get_CircleInFocPlaneFromPts(DTYPE_t O0, DTYPE_t O1, DTYPE_t O2, DTYPE_t nIn0, DTYPE_t nIn1, DTYPE_t nIn2, DTYPE_t Rad, DTYPE_t F1, 
        np.ndarray[DTYPE_t, ndim=1, mode='c'] Pts0, np.ndarray[DTYPE_t, ndim=1, mode='c'] Pts1, np.ndarray[DTYPE_t, ndim=1, mode='c'] Pts2, DTYPE_t F2=np.inf):
    """
    Return the center and radius of the image of the spherical Lens projected in its focal plane as seen from arbitrary points, only usable with F2=inf so far
    
    Inputs:
    -------
        O       iterable    Contains the 3D cartesian coordinates of the center of the Lens
        nIn     iterable    Contains the 3D cartesian coordinates of the normalized vector indicating the axis of the Lens, directing towards the inside of the vessel
        Rad     float       Radius of the Lens
        F1      float       Focal distance (positive) on the side of the detector (i.e.: opposite side of the plasma and points of interest)
        Pts     iterable    Contains the 3D cartesian coordinates of all the N points for which the computation shall be done
        F2      float       inf

    Outputs:
    --------
        Cs      np.ndarray  (3,N) array of the 3D cartesian coordiantes of the centers of the circular images of the spherical Lens in its focal plane as seen from the N Pts
        RadIm   np.ndarray  (N,) array of the radius of each circular image of the Lens seen from the N points
        din     np.ndarray  (N,) array of the distance from the Lens center O of the N points along the axis (O,nIn)
        r       np.ndarray  (N,) array of the (absolute, positive) distance to the axis (O,nIn) of the N Pts
        rIm     np.ndarray  (N,) array of the (absolute, positive) distance to the axis (O,nIn) of the N centers Cs of the Lens images
        nperp   np.ndarray  (3,N) array of the 3D cartesian coordinates of the normalized vector perpendicular to the axis (O,nIn) and passing throught the N Pts
    """
    assert F2==np.inf, "Only coded for F2==inf !"

    # Computing the algorithmic distance between the Lens center O and each point along the Lens axis directed by nIn
    cdef np.ndarray[DTYPE_t, ndim=1, mode='c'] din = (Pts0-O0)*nIn0 + (Pts1-O1)*nIn1 + (Pts2-O2)*nIn2
    # Deducing the normalised vector perpendicular to the Lens axis, and the distance r to the axis
    cdef np.ndarray[DTYPE_t, ndim=1, mode='c'] nperp0 = Pts0-O0-din*nIn0, nperp1 = Pts1-O1-din*nIn1, nperp2 = Pts2-O2-din*nIn2
    cdef np.ndarray[DTYPE_t, ndim=1, mode='c'] r = np.sqrt(nperp0**2+nperp1**2+nperp2**2)
    indokr, indokd = r>0., din>0.
    nperp0[indokr], nperp1[indokr], nperp2[indokr] = nperp0[indokr]/r[indokr], nperp1[indokr]/r[indokr], nperp2[indokr]/r[indokr]

    # Computing the distance rIm between the Lens axis and the center of the Lens image in the focal plane from Pts
    cdef np.ndarray[DTYPE_t, ndim=1, mode='c'] rIm = r * F1
    rIm[indokd] = rIm[indokd] / din[indokd]
    cdef np.ndarray[DTYPE_t, ndim=1, mode='c'] Cs0 = O0-F1*nIn0-nperp0*rIm, Cs1 = O1-F1*nIn1-nperp1*rIm, Cs2 = O2-F1*nIn2-nperp2*rIm

    # Computing the width the Lens image in the focal plane
    cdef np.ndarray[DTYPE_t, ndim=1, mode='c'] OC = np.sqrt((O0-Cs0)**2+(O1-Cs1)**2+(O2-Cs2)**2)
    cdef np.ndarray[DTYPE_t, ndim=1, mode='c'] OA = np.sqrt((O0-Pts0)**2+(O1-Pts1)**2+(O2-Pts2)**2)
    cdef np.ndarray[DTYPE_t, ndim=1, mode='c'] RadIm = Rad * OC/OA

    return Cs0,Cs1,Cs2, RadIm, din, r, rIm, nperp0,nperp1,nperp2

cdef Calc_SAngVect_LPolysPoints_Lens(DTYPE_t O0, DTYPE_t O1, DTYPE_t O2, DTYPE_t Tip0, DTYPE_t Tip1, DTYPE_t Tip2, DTYPE_t nIn0, DTYPE_t nIn1, DTYPE_t nIn2, np.ndarray[DTYPE_t, ndim=1,mode='c'] Pt0, np.ndarray[DTYPE_t, ndim=1,mode='c'] Pt1, np.ndarray[DTYPE_t, ndim=1,mode='c'] Pt2, DTYPE_t RadL, DTYPE_t RadD, DTYPE_t F1, DTYPE_t tanthetmax, np.ndarray[DTYPE_t, ndim=1,mode='c'] PolyL0, np.ndarray[DTYPE_t, ndim=1,mode='c'] PolyL1, np.ndarray[DTYPE_t, ndim=1,mode='c'] PolyL2, DTYPE_t[::1] thet=np.linspace(0.,np.pi,100)):
    """
    Return the solid angle and vector associated to one point in the plasma and corresponding to the intersection between the image of a spherical lens and a circular detector placed in its focal plane with the same axis
    """
    cdef int NP = Pt0.shape[0] #int(Pt0.size)
    cdef int NThet = int(thet.size)
    cdef np.ndarray[DTYPE_t, ndim=1,mode='c'] SAng = np.zeros((NP,))
    cdef np.ndarray[DTYPE_t, ndim=1,mode='c'] Vect0 = np.nan*np.ones((NP,)), Vect1 = np.nan*np.ones((NP,)), Vect2 = np.nan*np.ones((NP,))
    cdef np.ndarray[DTYPE_t, ndim=1,mode='c'] C0, C1, C2, RadIm, d, r, rIm, nperp0, nperp1, nperp2
    cdef np.ndarray[DTYPE_t, ndim=1,mode='c'] nIn = np.array([nIn0, nIn1, nIn2])
    cdef np.ndarray[DTYPE_t, ndim=1,mode='c'] e1, e2, norm
    cdef DTYPE_t Ref0, Ref1, Ref2
    cdef np.ndarray[DTYPE_t, ndim=1,mode='c'] Rbis
    cdef DTYPE_t[::1] polyL0, polyL1, polyL2, poly0, poly1
    cdef DTYPE_t BaryS0, BaryS1
    cdef DTYPE_t AreaLim = math.pi*(min(RadL,RadD)/10000.)**2

    ind = LensDetect_isInside(O0,O1,O2, Tip0,Tip1,Tip2, nIn0,nIn1,nIn2, Pt0,Pt1,Pt2, tanthetmax)
    if np.any(ind):     # Check point has non-zero Solid Angle (is inside the viewing cone)
        Ind = ind.nonzero()[0]

        C0,C1,C2, RadIm, d, r, rIm, nperp0,nperp1,nperp2 = _Lens_get_CircleInFocPlaneFromPts(O0,O1,O2, nIn0,nIn1,nIn2, RadL, F1, Pt0[ind],Pt1[ind],Pt2[ind])
        e1 = np.array([nperp0[0],nperp1[0],nperp2[0]])
        e1 = e1/np.linalg.norm(e1)
        e2 = np.array([nIn1*nperp2[0]-nIn2*nperp1[0], nIn2*nperp0[0]-nIn0*nperp2[0], nIn0*nperp1[0]-nIn1*nperp0[0]])
        e2 = e2/np.linalg.norm(e2)

        indrIm = rIm < RadIm+RadD
        if np.any(indrIm): # Make sure the intersection exists
            
            indrIm1 = rIm+RadIm <= RadD          # Case with image of Lens included in Detector
            indrIm2 = RadIm >= rIm+RadD          # Case with Detector included in image of Lens
            indrIm3 = ~(indrIm1 | indrIm2)       # The rest

            if np.any(indrIm1):
                RefP = np.array([O0, O1, O2])
                SAng[Ind[indrIm1]] = Calc_SAngVect_LPolysPoints([np.array([PolyL0,PolyL1,PolyL2])], np.array([Pt0[ind][indrIm1],Pt1[ind][indrIm1],Pt2[ind][indrIm1]]), RefP, nIn, e1, e2)[0]
                Vect0[Ind[indrIm1]], Vect1[Ind[indrIm1]], Vect2[Ind[indrIm1]] = RefP[0]-Pt0[ind][indrIm1], RefP[1]-Pt1[ind][indrIm1], RefP[2]-Pt2[ind][indrIm1]
            
            if np.any(indrIm2):
                RefP =  np.array([O0+nperp0[indrIm2]*r[indrIm2], O1+nperp1[indrIm2]*r[indrIm2], O2+nperp2[indrIm2]*r[indrIm2]]) # -nIn ?
                Rbis = RadD * d[indrIm2] / F1
                IndrIm2 = indrIm2.nonzero()[0]
                for ii in range(0,indrIm2.sum()):
                    e20, e21, e22 = nIn1*nperp2[IndrIm2[ii]]-nIn2*nperp1[IndrIm2[ii]], nIn2*nperp0[IndrIm2[ii]]-nIn0*nperp2[IndrIm2[ii]], nIn0*nperp1[IndrIm2[ii]]-nIn1*nperp0[IndrIm2[ii]]
                    polyL0 = RefP[0,ii] + Rbis[ii]*np.cos(thet)*nperp0[IndrIm2[ii]] + Rbis*np.sin(thet)*e20
                    polyL1 = RefP[1,ii] + Rbis[ii]*np.cos(thet)*nperp1[IndrIm2[ii]] + Rbis*np.sin(thet)*e21
                    polyL2 = RefP[2,ii] + Rbis[ii]*np.cos(thet)*nperp2[IndrIm2[ii]] + Rbis*np.sin(thet)*e22
                    SAng[Ind[IndrIm2[ii]]] = Calc_SAngVect_1Point1Poly_NoVect(Pt0[ind][IndrIm2[ii]],Pt1[ind][IndrIm2[ii]],Pt2[ind][IndrIm2[ii]], polyL0,polyL1,polyL2, RefP[0,ii],RefP[1,ii],RefP[2,ii])
                    Vect0[Ind[IndrIm2[ii]]], Vect1[Ind[IndrIm2[ii]]], Vect2[Ind[IndrIm2[ii]]] = RefP[0,ii]-Pt0[ind][IndrIm2[ii]], RefP[1,ii]-Pt1[ind][IndrIm2[ii]], RefP[2,ii]-Pt2[ind][IndrIm2[ii]]

            if np.any(indrIm3):
                Rbis = RadD * d[indrIm3] / F1
                IndrIm3 = indrIm3.nonzero()[0]
                for ii in range(0,indrIm3.sum()):
                    e20, e21, e22 = nIn1*nperp2[IndrIm3[ii]]-nIn2*nperp1[IndrIm3[ii]], nIn2*nperp0[IndrIm3[ii]]-nIn0*nperp2[IndrIm3[ii]], nIn0*nperp1[IndrIm3[ii]]-nIn1*nperp0[IndrIm3[ii]]
                    polyL0 = (PolyL0-O0)*nperp0[IndrIm3[ii]] + (PolyL1-O1)*nperp1[IndrIm3[ii]] + (PolyL2-O2)*nperp2[IndrIm3[ii]]
                    polyL1 = (PolyL0-O0)*e20 + (PolyL1-O1)*e21 + (PolyL2-O2)*e22
                    poly0 = r[IndrIm3[ii]] + Rbis[ii]*np.cos(thet)
                    poly1 = Rbis[ii]*np.sin(thet)
                    PolyInt = plg.Polygon(np.array([polyL0,polyL1]).T) & plg.Polygon(np.array([poly0,poly1]).T)
                    if PolyInt.area() > AreaLim:
                        BaryS0, BaryS1 = PolyInt.center()
                        Ref0, Ref1, Ref2 = O0+BaryS0*nperp0[IndrIm3[ii]]+BaryS1*e20, O1+BaryS0*nperp1[IndrIm3[ii]]+BaryS1*e21, O2+BaryS0*nperp2[IndrIm3[ii]]+BaryS1*e22
                        PolyInt0, PolyInt1 = np.array(PolyInt[0]).T
                        PolyLL0 = O0 + PolyInt0*nperp0[IndrIm3[ii]]+PolyInt1*e20
                        PolyLL1 = O1 + PolyInt0*nperp1[IndrIm3[ii]]+PolyInt1*e21
                        PolyLL2 = O2 + PolyInt0*nperp2[IndrIm3[ii]]+PolyInt1*e22
                        SAng[Ind[IndrIm3[ii]]] = Calc_SAngVect_1Point1Poly_NoVect(Pt0[ind][IndrIm3[ii]],Pt1[ind][IndrIm3[ii]],Pt2[ind][IndrIm3[ii]], PolyLL0,PolyLL1,PolyLL2, Ref0,Ref1,Ref2)
                        Vect0[Ind[IndrIm3[ii]]], Vect1[Ind[IndrIm3[ii]]], Vect2[Ind[IndrIm3[ii]]] = Ref0-Pt0[ind][IndrIm3[ii]], Ref1-Pt1[ind][IndrIm3[ii]], Ref2-Pt2[ind][IndrIm3[ii]]

            norm = np.sqrt(Vect0**2+Vect1**2+Vect2**2)
            Vect0, Vect1, Vect2 = Vect0/norm, Vect1/norm, Vect2/norm

    return SAng, np.array([Vect0,Vect1,Vect2])